In [1]:
!pip install langchain-google-genai
!pip install google-search-results
!pip install langchain langchain_community
!pip install langgraph faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 36.4 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32010 sha256=2ed7cc734296f804059b7398ee90f9ca5c54189da7a3bf8e9d01e87a344e4448
  Stored in directory: /root/.cache/pip/wheels/6e/42/3e/aeb691b02cb7175ec70e2da04b5658d4739d2b41e5f73cd06f
Successfully built google-search-results
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.2/153.2 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 5.2 MB/s eta 0:00:00
  

In [2]:
import os
import json
import time
from typing import List, Dict, Any, Optional
from io import BytesIO
import base64
import matplotlib.pyplot as plt
from urllib.parse import urlparse

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from serpapi import GoogleSearch

# ----- LangGraph -----
from langgraph.graph import StateGraph, START, END
from typing import TypedDict

# ----- FAISS -----
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document

# -------------------------------
# Critic Agent with Fact-Checking
# -------------------------------
class CriticAgent:
    def __init__(self, google_api_key: str):
        self.llm = ChatGoogleGenerativeAI(
            model="gemini-1.5-flash",
            temperature=0,
            google_api_key=google_api_key
        )
        self.prompt_template = PromptTemplate(
            input_variables=["facts"],
            template="""
You are a fact-checking agent. Your task is to evaluate each fact for accuracy based on general knowledge and reputable information sources.

Facts to review:
{facts}

For each fact, return a JSON list with:
- fact: The original fact (keep it short).
- source: The source URL (if provided with the fact).
- reliability_score: Integer 0–5 (5 = very reliable, 0 = false/misleading).
- reasoning: A brief 1–2 sentence explanation.

Return ONLY valid JSON in the following format:
[
  {{ "fact": "...", "source": "https://...", "reliability_score": 5, "reasoning": "..." }},
  ...
]
            """
        )

    def evaluate_facts(self, facts: List[Dict[str, Any]]):
        facts_str = json.dumps(facts, indent=2)
        chain = LLMChain(llm=self.llm, prompt=self.prompt_template)
        result = chain.run(facts=facts_str)

        try:
            json_string = result.strip()
            if json_string.startswith("```json"):
                json_string = json_string.replace("```json", "").replace("```", "").strip()

            parsed = json.loads(json_string)
            if not isinstance(parsed, list):
                raise ValueError("CriticAgent output is not a JSON list.")

            # Drop facts with reliability_score < 3
            filtered = [f for f in parsed if f.get("reliability_score", 0) >= 3]
            return filtered

        except (json.JSONDecodeError, ValueError) as e:
            return {
                "error": "Failed to parse JSON from CriticAgent",
                "raw_output": result,
                "details": str(e)
            }

# -------------------------------
# Research Agent
# -------------------------------
class ResearchAgent:
    def __init__(self, serpapi_key: str, critic_agent: CriticAgent, google_api_key: str):
        self.serpapi_key = serpapi_key
        self.critic_agent = critic_agent
        self.llm = ChatGoogleGenerativeAI(
            model="gemini-1.5-flash",
            temperature=0,
            google_api_key=google_api_key
        )
        self.prompt_template = PromptTemplate(
            input_variables=["query", "search_results"],
            template="""
You are a Research Agent.
User query: "{query}"

Search results:
{search_results}

From these results, extract the 5 most important factual statements.
Each fact should be short (max 25 words) and tied to its source link.

Return strictly JSON:
[
  {{ "fact": "string", "source": "string" }},
  ...
]
            """
        )

    def serpapi_search(self, query: str):
        search = GoogleSearch({
            "q": query,
            "api_key": self.serpapi_key,
            "num": 10
        })
        results = search.get_dict()
        return results.get("organic_results", [])

    def get_domain_from_url(self, url: str) -> Optional[str]:
        try:
            return urlparse(url).netloc
        except:
            return None

    def clean_and_score_facts(self, facts, raw_results):
        cleaned = []
        seen = set()
        result_rank_map = {res.get("link"): res.get("position") for res in raw_results if res.get("link")}

        trusted_domains = set()
        for res in raw_results[:5]:
            domain = self.get_domain_from_url(res.get("link", ""))
            if domain:
                trusted_domains.add(domain)

        for item in facts:
            fact = item.get("fact", "").strip()
            source = item.get("source", "").strip()
            source_domain = self.get_domain_from_url(source)

            if not fact or fact.lower() in seen:
                continue
            seen.add(fact.lower())

            score = 0
            if len(fact) > 20:
                score += 1
            if any(char.isdigit() for char in fact):
                score += 1
            if source_domain and source_domain in trusted_domains:
                score += 1
            rank = result_rank_map.get(source)
            if rank:
                if rank <= 3:
                    score += 2
                elif rank <= 7:
                    score += 1
            if score > 2:
                cleaned.append({
                    "fact": fact,
                    "source": source,
                    "confidence_score": score,
                    "rank": rank
                })

        return cleaned

    def run(self, query: str):
        raw_results = self.serpapi_search(query)
        search_results_text = []
        for res in raw_results:
            title = res.get("title", "")
            snippet = res.get("snippet", "")
            link = res.get("link", "")
            search_results_text.append(f"{title} — {snippet} (URL: {link})")

        chain = LLMChain(llm=self.llm, prompt=self.prompt_template)
        structured_facts_str = chain.run(query=query, search_results="\n".join(search_results_text))

        try:
            json_string = structured_facts_str.strip()
            if json_string.startswith("```json"):
                json_string = json_string.replace("```json", "").replace("```", "").strip()
            facts = json.loads(json_string)
            if not isinstance(facts, list):
                raise ValueError("LLM output is not a JSON list.")
            for item in facts:
                if not isinstance(item, dict) or "fact" not in item or "source" not in item:
                    raise ValueError("LLM output list items are not properly formatted.")
        except (json.JSONDecodeError, ValueError) as e:
            return {"error": "Failed to parse or validate JSON from LLM", "raw_output": structured_facts_str, "details": str(e)}

        high_value_facts = self.clean_and_score_facts(facts, raw_results)
        validated_facts = self.critic_agent.evaluate_facts(high_value_facts)
        return validated_facts

# -------------------------------
# Writer Agent
# -------------------------------
class WriterAgent:
    def __init__(self, google_api_key: str):
        self.llm = ChatGoogleGenerativeAI(
            model="gemini-1.5-flash",
            temperature=0.5,
            google_api_key=google_api_key
        )
        self.summary_prompt = PromptTemplate(
            input_variables=["facts"],
            template="""
You are a Writer Agent.
Turn the validated facts below into a clear, human-friendly executive summary for a general audience.
Add a concise 'Key Takeaways' section with bullet points.

Facts:
{facts}

Return strictly JSON in the format:
{{
  "executive_summary": "string",
  "key_takeaways": ["point1", "point2", ...]
}}
            """
        )

    def generate_summary(self, facts: List[Dict[str, Any]]):
        facts_str = json.dumps(facts, indent=2)
        chain = LLMChain(llm=self.llm, prompt=self.summary_prompt)
        result = chain.run(facts=facts_str)

        try:
            json_string = result.strip()
            if json_string.startswith("```json"):
                json_string = json_string.replace("```json", "").replace("```", "").strip()
            parsed = json.loads(json_string)
            return parsed
        except (json.JSONDecodeError, ValueError) as e:
            return {
                "error": "Failed to parse JSON from WriterAgent",
                "raw_output": result,
                "details": str(e)
            }

    def create_visualization(self, facts: List[Dict[str, Any]]) -> Optional[str]:
        """Create a simple bar chart of reliability scores per fact; returns data URI."""
        try:
            labels = []
            scores = []
            for f in facts:
                t = f.get("fact", "")
                labels.append(t[:50] + "..." if len(t) > 50 else t)
                scores.append(f.get("reliability_score", 0))

            if not labels:
                return None

            plt.figure(figsize=(8, 4))
            plt.barh(labels, scores)
            plt.xlabel("Reliability Score")
            plt.title("Fact Reliability Overview")
            plt.tight_layout()

            buf = BytesIO()
            plt.savefig(buf, format="png")
            plt.close()
            buf.seek(0)
            img_base64 = base64.b64encode(buf.read()).decode("utf-8")
            return f"data:image/png;base64,{img_base64}"
        except Exception:
            return None

# -------------------------------
# Planner Agent
# -------------------------------
class PlannerAgent:
    def __init__(self, google_api_key: str):
        self.llm = ChatGoogleGenerativeAI(
            model="gemini-1.5-flash",
            temperature=0.7,
            google_api_key=google_api_key
        )
        self.plan_prompt = PromptTemplate(
            input_variables=["facts", "summary"],
            template="""
You are a Planner Agent.
Based on the validated facts and the executive summary, identify:
1. Gaps in the research
2. Open questions
3. Related subtopics worth exploring

Prioritize these suggestions based on:
- Relevance to the original query
- Importance to understanding the topic
- Feasibility of researching them further

Return strictly JSON in the format:
{{
  "gaps_identified": ["gap1", "gap2", ...],
  "open_questions": ["question1", "question2", ...],
  "recommended_next_steps": [
    {{ "step": "string", "priority": "High|Medium|Low" }}
  ]
}}
            """
        )

    def suggest_next_steps(self, facts: List[Dict[str, Any]], summary: Dict[str, Any]):
        facts_str = json.dumps(facts, indent=2)
        summary_str = json.dumps(summary, indent=2)
        chain = LLMChain(llm=self.llm, prompt=self.plan_prompt)
        result = chain.run(facts=facts_str, summary=summary_str)

        try:
            json_string = result.strip()
            if json_string.startswith("```json"):
                json_string = json_string.replace("```json", "").replace("```", "").strip()
            parsed = json.loads(json_string)
            return parsed
        except (json.JSONDecodeError, ValueError) as e:
            return {
                "error": "Failed to parse JSON from PlannerAgent",
                "raw_output": result,
                "details": str(e)
            }

# ==========================================================
#                  FAISS Persistence Helpers
# ==========================================================
FAISS_DIR = "faiss_index"

def get_embeddings(google_api_key: str):
    return GoogleGenerativeAIEmbeddings(
        model="models/text-embedding-004",
        google_api_key=google_api_key
    )

def load_or_create_faiss(embeddings) -> FAISS:
    if os.path.isdir(FAISS_DIR):
        try:
            return FAISS.load_local(FAISS_DIR, embeddings, allow_dangerous_deserialization=True)
        except Exception:
            pass

    dummy_text = "This is a dummy text."
    dummy_embedding = embeddings.embed_query(dummy_text)
    embedding_dimension = len(dummy_embedding)

    from faiss import IndexFlatL2
    index = IndexFlatL2(embedding_dimension)
    from langchain_community.docstore import InMemoryDocstore
    from uuid import uuid4
    empty_docs = []
    docstore = InMemoryDocstore({str(uuid4()): Document(page_content="") for _ in range(0)})
    index_to_docstore_id = {}

    return FAISS(embeddings.embed_query, index, docstore, index_to_docstore_id)


def persist_facts_to_faiss(vs: FAISS, facts: List[Dict[str, Any]]):
    texts = []
    metas = []
    now = int(time.time())
    for f in facts:
        # Compose a retrievable text: fact + source
        fact_text = f.get("fact", "")
        source = f.get("source") or f.get("source_url") or ""
        full_text = f"{fact_text}\nSOURCE: {source}"
        texts.append(full_text)
        metas.append({
            "source": source,
            "reliability_score": f.get("reliability_score"),
            "confidence_score": f.get("confidence_score"),
            "rank": f.get("rank"),
            "timestamp": now
        })
    if texts:
        vs.add_texts(texts=texts, metadatas=metas)
        vs.save_local(FAISS_DIR)

def retrieve_similar_facts(vs: FAISS, query: str, k: int = 5):
    if vs is None:
        return []
    docs = vs.similarity_search(query, k=k)
    return [{"content": d.page_content, "metadata": d.metadata} for d in docs]

# ==========================================================
#                  LangGraph
# ==========================================================
class GraphState(TypedDict, total=False):
    query: str
    validated_facts: List[Dict[str, Any]]
    executive_summary: Dict[str, Any]
    visualization_data_uri: Optional[str]
    plan: Dict[str, Any]
    retrieved_context: List[Dict[str, Any]]

def build_graph(researcher: ResearchAgent,
                writer: WriterAgent,
                planner: PlannerAgent,
                embeddings,
                ):

    vectorstore = load_or_create_faiss(embeddings)

    def node_research(state: GraphState) -> GraphState:
        validated = researcher.run(state["query"])
        return {"validated_facts": validated}

    def node_persist_and_retrieve(state: GraphState) -> GraphState:
        facts = state.get("validated_facts", [])
        if isinstance(facts, list) and all(isinstance(f, dict) for f in facts):
             persist_facts_to_faiss(vectorstore, facts)

        retrieved = retrieve_similar_facts(vectorstore, state["query"], k=5)
        return {"retrieved_context": retrieved}


    def node_write(state: GraphState) -> GraphState:
        facts = state.get("validated_facts", [])
        summary = writer.generate_summary(facts)
        viz = writer.create_visualization(facts)
        return {"executive_summary": summary, "visualization_data_uri": viz}

    def node_plan(state: GraphState) -> GraphState:
        facts = state.get("validated_facts", [])
        summary = state.get("executive_summary", {})
        plan = planner.suggest_next_steps(facts, summary)
        return {"plan": plan}

    graph = StateGraph(GraphState)
    graph.add_node("research", node_research)
    graph.add_node("persist_and_retrieve", node_persist_and_retrieve)
    graph.add_node("writer", node_write)
    graph.add_node("planner", node_plan)

    graph.add_edge(START, "research")
    graph.add_edge("research", "persist_and_retrieve")
    graph.add_edge("persist_and_retrieve", "writer")
    graph.add_edge("writer", "planner")
    graph.add_edge("planner", END)

    return graph.compile()

In [3]:
# ==========================================================
#                        Runner
# ==========================================================
if __name__ == "__main__":
    SERPAPI_KEY = "SerpAPI-key"
    GOOGLE_API_KEY = "GoogleAPI-key"

    # Agents
    critic = CriticAgent(GOOGLE_API_KEY)
    researcher = ResearchAgent(SERPAPI_KEY, critic, GOOGLE_API_KEY)
    writer = WriterAgent(GOOGLE_API_KEY)
    planner = PlannerAgent(GOOGLE_API_KEY)

    # Embeddings (langchain[google-genai])
    embeddings = get_embeddings(GOOGLE_API_KEY)

    # Build & run graph
    app = build_graph(researcher, writer, planner, embeddings)

    initial_state: GraphState = {
        "query": "Summarize the Knowledge distillation from this link: http://arxiv.org/pdf/2504.05299"
        #"query": "Knowledge Distillation VLM"
    }
    final_state = app.invoke(initial_state)

    print("\n=== EXECUTIVE SUMMARY ===")
    es = final_state.get("executive_summary", {})
    print(es.get("executive_summary", ""))
    print("\n=== KEY TAKEAWAYS ===")
    for t in es.get("key_takeaways", []):
        print(f"- {t}")

    print("\n=== NEXT RESEARCH PLAN ===")
    print(json.dumps(final_state.get("plan", {}), indent=2))

    if final_state.get("visualization_data_uri"):
        print("\nVisualization (base64 preview):")
        print(final_state["visualization_data_uri"][:120] + "...")

/tmp/ipython-input-891063002.py:179: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=self.llm, prompt=self.prompt_template)
/tmp/ipython-input-891063002.py:180: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  structured_facts_str = chain.run(query=query, search_results="\n".join(search_results_text))



=== EXECUTIVE SUMMARY ===
Knowledge distillation is a powerful technique for improving and streamlining acoustic models.  It works by transferring the knowledge learned by a large ensemble of models (the 'teacher') into a smaller, more efficient single model (the 'student'). This process allows for significant reductions in model size without necessarily sacrificing performance.  Research shows that various knowledge categories, training methods, and teacher-student relationships are key factors influencing the success of this technique. While not always guaranteed to maintain performance perfectly, when implemented correctly, knowledge distillation offers a promising path to creating faster, more deployable acoustic models.

=== KEY TAKEAWAYS ===
- Knowledge distillation shrinks large acoustic models into smaller, faster ones.
- It achieves this by transferring knowledge from a group of models to a single model.
- Success depends on the type of knowledge transferred, training methods